In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA


In [2]:
# -----------------------------
# 1) Paths & output folder
# -----------------------------
DATA_PATH = "data.xlsx"
LABEL_PATH = "labels.xlsx"
OUT_DIR = "dataset_stats_plots"
os.makedirs(OUT_DIR, exist_ok=True)

In [3]:
# -----------------------------
# 2) Read data
# -----------------------------
# data.xlsx: shape = (bands, samples)
data_df = pd.read_excel(DATA_PATH, header=None)
# labels.xlsx: shape = (samples, 1)
labels_df = pd.read_excel(LABEL_PATH, header=None)

# Convert to useful shapes
# X: (samples, bands)
X = data_df.values.T
y = labels_df.iloc[:, 0].values  # raw labels (original units)

n_samples, n_bands = X.shape
print(f"Number of samples : {n_samples}")
print(f"Number of bands   : {n_bands}")

# Eğer dalga boylarını 200–900 nm aralığına eşit aralıklarla yaymak istersen:
wavelengths = np.linspace(200, 900, n_bands)

Number of samples : 152
Number of bands   : 3501


In [4]:
# -----------------------------
# 3) Label statistics
# -----------------------------
labels_series = pd.Series(y, name="phosphorus")

label_stats = labels_series.describe()
print("\nLabel statistics :")
print(label_stats)

# Save label stats as Excel
label_stats.to_frame().to_excel(os.path.join(OUT_DIR, "label_statistics.xlsx"))

# Histogram of labels
plt.figure(figsize=(6, 4))
plt.hist(y, bins=15, edgecolor="black")
plt.xlabel("Phosphorus (mg/kg)")
plt.ylabel("Count")
plt.title("Histogram of soil phosphorus values")
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "label_histogram.png"), dpi=300)
plt.close()

# Boxplot of labels
plt.figure(figsize=(4, 5))
plt.boxplot(y, vert=True, showmeans=True)
plt.ylabel("Phosphorus (mg/kg)")
plt.title("Boxplot of soil phosphorus values")
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "label_boxplot.png"), dpi=300)
plt.close()


Label statistics :
count       152.000000
mean     109052.320101
std       64700.890091
min         400.300000
25%       59936.712940
50%       96197.784875
75%      170426.473813
max      212571.000000
Name: phosphorus, dtype: float64


In [5]:

# -----------------------------
# 4) Spectral statistics
# -----------------------------
# Per-band statistics across samples
spectral_df = pd.DataFrame(X, columns=[f"band_{i}" for i in range(n_bands)])
spec_stats = spectral_df.describe().T[["mean", "std", "min", "max"]]
spec_stats.index = wavelengths  # index as wavelength (nm)
spec_stats.index.name = "wavelength_nm"

print("\nSpectral statistics (per band, mean/std/min/max) – first 5 rows:")
print(spec_stats.head())

# Save spectral stats as Excel
spec_stats.to_excel(os.path.join(OUT_DIR, "spectral_statistics_by_band.xlsx"))


Spectral statistics (per band, mean/std/min/max) – first 5 rows:
                    mean        std       min    max
wavelength_nm                                       
200.0          49.731044  16.892542  3.115585  100.0
200.2          51.270806  18.146585  0.000000  100.0
200.4          52.784951  20.166044  5.149837  100.0
200.6          54.986670  22.315968  1.694811  100.0
200.8          51.620816  21.517827  7.682172  100.0


In [6]:
# -----------------------------
# 5) Plot: first 10 spectra
# -----------------------------
plt.figure(figsize=(8, 5))
num_to_plot = min(10, n_samples)
for i in range(num_to_plot):
    plt.plot(wavelengths, X[i, :], alpha=0.7)
plt.xlabel("Wavelength (nm)")
plt.ylabel("Reflectance")
plt.title(f"Example reflectance spectra (first {num_to_plot} samples)")
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "example_spectra_first10.png"), dpi=300)
plt.close()

In [7]:
# -----------------------------
# 6) Plot: mean ± std spectrum
# -----------------------------
mean_spectrum = X.mean(axis=0)
std_spectrum = X.std(axis=0)

plt.figure(figsize=(8, 5))
plt.plot(wavelengths, mean_spectrum, label="Mean reflectance")
plt.fill_between(
    wavelengths,
    mean_spectrum - std_spectrum,
    mean_spectrum + std_spectrum,
    alpha=0.3,
    label="±1 std"
)
plt.xlabel("Wavelength (nm)")
plt.ylabel("Reflectance")
plt.title("Mean spectrum with ±1 standard deviation")
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "mean_spectrum_with_std.png"), dpi=300)
plt.close()

print(f"\nAll statistics and plots saved in folder: {OUT_DIR}")


All statistics and plots saved in folder: dataset_stats_plots


In [8]:
# -----------------------------
# 7) Band–label correlation vs wavelength
# -----------------------------
# Pearson korelasyonu: her band ile fosfor etiketi arasındaki ilişki
corrs = []
for j in range(n_bands):
    x_band = X[:, j]
    # np.corrcoef: 2x2 korelasyon matrisi döner, [0,1] veya [1,0] off-diagonal eleman korelasyon
    r = np.corrcoef(x_band, y)[0, 1]
    corrs.append(r)

corrs = np.array(corrs)

# Korelasyonları tablo olarak da kaydedelim
corr_df = pd.DataFrame({
    "wavelength_nm": wavelengths,
    "corr_with_P": corrs
})
corr_df.to_excel(os.path.join(OUT_DIR, "band_label_correlation.xlsx"), index=False)

# Plot: correlation vs wavelength
plt.figure(figsize=(8, 5))
plt.plot(wavelengths, corrs)
plt.axhline(0.0, linestyle="--", linewidth=1)
plt.xlabel("Wavelength (nm)")
plt.ylabel("Correlation with phosphorus")
plt.title("Band–label Pearson correlation across wavelengths")
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "band_label_correlation_curve.png"), dpi=300)
plt.close()


In [9]:
# -----------------------------
# 8) Low vs high P mean spectra
# -----------------------------
# Alt %25 → low P, üst %25 → high P
q25 = np.percentile(y, 25)
q75 = np.percentile(y, 75)

low_mask = y <= q25
high_mask = y >= q75

X_low = X[low_mask]
X_high = X[high_mask]

low_mean = X_low.mean(axis=0)
high_mean = X_high.mean(axis=0)

low_std = X_low.std(axis=0)
high_std = X_high.std(axis=0)

print(f"\nLow-P samples:  n = {low_mask.sum()}, threshold ≤ {q25:.3f}")
print(f"High-P samples: n = {high_mask.sum()}, threshold ≥ {q75:.3f}")

plt.figure(figsize=(8, 5))
plt.plot(wavelengths, low_mean, label="Low P (mean)")
plt.fill_between(
    wavelengths,
    low_mean - low_std,
    low_mean + low_std,
    alpha=0.2,
    label="Low P ±1 std"
)
plt.plot(wavelengths, high_mean, label="High P (mean)")
plt.fill_between(
    wavelengths,
    high_mean - high_std,
    high_mean + high_std,
    alpha=0.2,
    label="High P ±1 std"
)
plt.xlabel("Wavelength (nm)")
plt.ylabel("Reflectance")
plt.title("Mean spectra of low- and high-phosphorus samples")
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "low_vs_high_P_mean_spectra.png"), dpi=300)
plt.close()



Low-P samples:  n = 38, threshold ≤ 59936.713
High-P samples: n = 38, threshold ≥ 170426.474


In [10]:
# -----------------------------
# 9) PCA scree plot & PC1–PC2 scatter (colored by P)
# -----------------------------
# PCA otomatik olarak veriyi ortalama etrafında merkezler
pca = PCA()
X_pca = pca.fit_transform(X)
expl_var = pca.explained_variance_ratio_

# (a) Scree plot: ilk 20 bileşeni gösterelim (veya daha azsa n_components)
max_comps = min(20, len(expl_var))

plt.figure(figsize=(7, 4))
plt.plot(np.arange(1, max_comps + 1), expl_var[:max_comps], marker="o")
plt.xlabel("Principal component")
plt.ylabel("Explained variance ratio")
plt.title("PCA scree plot (first components)")
plt.xticks(np.arange(1, max_comps + 1))
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "pca_scree_plot.png"), dpi=300)
plt.close()

# (b) PC1 vs PC2 scatter, renk = gerçek fosfor değeri
plt.figure(figsize=(6, 5))
sc = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y, s=25, alpha=0.8)
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title("PCA scatter (PC1 vs PC2, colored by phosphorus)")
cbar = plt.colorbar(sc)
cbar.set_label("Phosphorus (mg/kg)")
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "pca_pc1_pc2_scatter_Pcolormap.png"), dpi=300)
plt.close()
